In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import os, time, itertools, pickle, random, glob, imageio
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt
import math

In [3]:
def dataset_load_album(datapath, labelpath, imgsize):
    """
    datapath 'jpeg' file of album image
    labelpath 'png' file of spectrogram 
    """
    datalist = os.listdir(datapath)
    datasize = len(datalist)
    
    data_ = np.zeros((datasize, imgsize, imgsize, 3))
    label_ = np.zeros((datasize, imgsize, imgsize, 3))
    nonelist = []
    for i, fname in enumerate(datalist):
        if glob.glob(os.path.join(labelpath, fname[:-4] + '*')):
        #f os.path.isfile(os.path.join(labelpath, fname[:-4] + 'png')):
            #print('there is file')
            img_d = Image.open(os.path.join(datapath, fname)
                              ).resize((imgsize, imgsize))
            img_l = Image.open(os.path.join(labelpath, fname[:-4]+'png')
                              ).convert('RGB').resize((imgsize, imgsize))
            data_[i] = np.asarray(img_d)
            label_[i] = np.asarray(img_l)
            img_d.close()
            img_l.close()
            
            
        else:
            nonelist.append(i)
            
    data = np.delete(data_, nonelist, 0)
    label = np.delete(label_, nonelist, 0)
    return data, label

In [4]:
# Load albumdata

img_size = 128

datapath = os.path.join('input', 'img')
audiopath = os.path.join('audio','spec')

samples, labels = dataset_load_album(
    datapath, audiopath, img_size)

samples = samples/255
labels = labels/255

input_dim = img_size * img_size * 3
num_sample = samples.shape[0]

In [21]:
class VariantionalAutoencoder(object):

    def __init__(self, sess, input_dim =input_dim, learning_rate=1e-4, batch_size=64, n_z=16):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z = n_z
        self.input_dim = input_dim

        self.sess = sess
        self.build()
        



    # Build the netowrk and the loss functions
    def build(self):
        
        self.x = tf.placeholder(
            name='x', dtype=tf.float32, shape=[None, self.input_dim])

        # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, 2048, scope='enc_fc1', activation_fn=tf.nn.relu)
        f2 = fc(f1, 1024, scope='enc_fc2', activation_fn=tf.nn.relu)
        f3 = fc(f2, 512, scope='enc_fc3', activation_fn=tf.nn.relu)
        f4 = fc(f3, 256, scope='enc_fc4', activation_fn=tf.nn.relu)
        f5 = fc(f4, 128, scope='enc_fc5', activation_fn=tf.nn.relu)
        f6 = fc(f5, 64, scope='enc_fc6', activation_fn=tf.nn.relu)
        self.z_mu = fc(f6, self.n_z, scope='enc_fc7_mu', 
                       activation_fn=None)
        self.z_log_sigma_sq = fc(f6, self.n_z, scope='enc_fc7_sigma', 
                                 activation_fn=None)
        eps = tf.random_normal(
            shape=tf.shape(self.z_log_sigma_sq),
            mean=0, stddev=1, dtype=tf.float32)
        self.z = self.z_mu + tf.sqrt(tf.exp(self.z_log_sigma_sq)) * eps

        # Decode
        # z -> x_hat
        g1 = fc(self.z, 64, scope='dec_fc1', activation_fn=tf.nn.relu)
        g2 = fc(g1, 128, scope='dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, 256, scope='dec_fc3', activation_fn=tf.nn.relu)
        g4 = fc(g3, 1024, scope='dec_fc4', activation_fn=tf.nn.relu)
        g5 = fc(g4, 1024, scope='dec_fc5', activation_fn=tf.nn.relu)
        g6 = fc(g5, 2048, scope='dec_fc6', activation_fn=tf.nn.relu)
        self.x_hat = fc(g6, self.input_dim, scope='dec_fc7', 
                        activation_fn=tf.sigmoid)

        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)
        epsilon = 1e-10
        recon_loss = -tf.reduce_sum(
            self.x * tf.log(epsilon+self.x_hat) + 
            (1-self.x) * tf.log(epsilon+1-self.x_hat), 
            axis=1
        )
        self.recon_loss = tf.reduce_mean(recon_loss)

        # Latent loss
        # KL divergence: measure the difference between two distributions
        # Here we measure the divergence between 
        # the latent distribution and N(0, 1)
        latent_loss = -0.5 * tf.reduce_sum(
            1 + self.z_log_sigma_sq - tf.square(self.z_mu) - 
            tf.exp(self.z_log_sigma_sq), axis=1)
        self.latent_loss = tf.reduce_mean(latent_loss)

        self.total_loss = self.recon_loss + self.latent_loss
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss)
        
        self.losses = {
            'recon_loss': self.recon_loss,
            'latent_loss': self.latent_loss,
            'total_loss': self.total_loss,
        }        
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x):
        _, losses = self.sess.run(
            [self.train_op, self.losses],
            feed_dict={self.x: x}
        )
        return losses

    # x -> x_hat
    def reconstructor(self, x, sess):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z

In [22]:
def trainer_album(model_object, sess, sample, input_dim =input_dim, learning_rate=1e-4, 
            batch_size=16, num_epoch=100, n_z=16, log_step=5,
                 num_sample = num_sample):
    sess.run(tf.global_variables_initializer())
    model = model_object(sess,
        learning_rate=learning_rate, batch_size=batch_size, n_z=n_z,
    input_dim =input_dim)
    
    
    step = 0

    for epoch in range(num_epoch):
        start_time = time.time()
        for iter in range(num_sample // batch_size):
            step += 1
            # Get a batch
            batch = sample[iter * batch_size : (iter + 1) * batch_size]
            # Execute the forward and backward pass 
            # Report computed losses
            #print('batch',batch)
            losses = model.run_single_step(batch)
        end_time = time.time()
        
        if epoch % log_step == 0:
            log_str = '[Epoch {}] '.format(epoch)
            for k, v in losses.items():
                log_str += '{}: {:.3f}  '.format(k, v)
            log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
            print(log_str)
            
    print('Done!')
    return model

In [23]:
tf.reset_default_graph()

with tf.Session(
    config=tf.ConfigProto(
        allow_soft_placement=True,log_device_placement=True)) as sess:
    with tf.device('/cpu:0'):
    #with tf.device('/GPU:0'): 
        sess.run(tf.global_variables_initializer())    
        model_2d_vae = trainer_album(VariantionalAutoencoder, sess,
                                 samples.reshape(-1,input_dim),
                                 input_dim =input_dim,
                                 batch_size=32, n_z=2, num_sample=samples.shape[0])
        saver = tf.train.Saver()
        saver.save(sess, "model.ckpt")

FailedPreconditionError: Attempting to use uninitialized value enc_fc7_sigma/biases
	 [[node enc_fc7_sigma/biases/read (defined at <ipython-input-21-7875f09ee181>:32) ]]

Caused by op 'enc_fc7_sigma/biases/read', defined at:
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Takato\Anaconda3\envs\cs599pro\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    

In [ ]:
# Test the trained model: generation
# Sample noise vectors from N(0, 1)
z = np.random.normal(size=[model_2d_vae.batch_size, model_2d_vae.n_z])
x_generated = model_2d_vae.generator(z)

n = np.sqrt(model_2d_vae.batch_size).astype(np.int32)
I_generated = np.empty((img_size*n, img_size*n, 3))
#print(x_generated.shape)
for i in range(n):
    #print(i)
    I_generated[i*img_size:(i+1)*img_size, i*img_size:(i+1)*img_size, :
               ] = x_generated[i].reshape(img_size, img_size, 3)

plt.figure(figsize=(8, 8))
plt.imshow(I_generated)

In [ ]:
test_transformation(model_2d_vae, mnist)

In [ ]:
# Test the trained model: uniformly samlpe in the latent space
n = 20
x = np.linspace(-2, 2, n)
y = np.linspace(-2, 2, n)

I_latent = np.empty((h*n, w*n))
for i, yi in enumerate(x):
    for j, xi in enumerate(y):
        z = np.array([[xi, yi]]*model_2d_vae.batch_size)
        x_hat = model_2d_vae.generator(z)
        I_latent[(n-i-1)*h:(n-i)*h, j*w:(j+1)*w] = x_hat[0].reshape(h, w)

plt.figure(figsize=(8, 8))        
plt.imshow(I_latent, cmap="gray")

In [ ]:
samples.shape

In [ ]:
samples.reshape(-1,32*32*3)[0]

In [ ]:
x_generated[0].reshape(32,32,3)